In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
import sys
import shap
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('https://github.com/jnin/information-systems/raw/main/data/compas_ai2.csv')

df['Severity'] = df['DecileScore'] > df['DecileScore'].median()
df.drop(columns = ['DecileScore'], inplace=True)

X = df.drop(columns=["Severity"])
y = df['Severity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

numerical_features = ['YearOfBirth', 'RecSupervisionLevel']

categorical_features = ['Agency','Gender','Ethnic','ScaleSet','LegalStatus','CustodyStatus',
                        'MaritalStatus','DisplayText']

In [ ]:
ohe = OneHotEncoder(sparse=False)

preprocessing_steps = [('transformed', ohe, categorical_features)]

transformer = ColumnTransformer(preprocessing_steps, remainder = 'passthrough')

pipeline_steps = [('preprocess', transformer),
                  ('scaler', StandardScaler()),
                  ('DT', DecisionTreeClassifier(random_state = 42))]

pipeline_steps

pipe = Pipeline(pipeline_steps)
pipe.fit(X_train, y_train)

In [ ]:
sample = pipe[0:2].transform(X_test)

explainer = shap.TreeExplainer(pipe[-1])

shap_values= explainer.shap_values(sample, approximate=True)

shap_values

In [ ]:
shap.initjs()
shap.summary_plot(shap_values, feature_names=pipe[0].get_feature_names_out(), plot_type="bar")

In [ ]:
shap.initjs()
shap.summary_plot(shap_values, feature_names=pipe[0].get_feature_names(), plot_type='bar')

In [ ]:
privileged = (X_test["Ethnic"] == "Caucasian")
favoured = ~pipe.predict(X_test)

In [ ]:
def compute_disparate_impact(privileged, favoured):
    
    pred_unfav = sum(~privileged * favoured) / sum(~privileged)
    pred_fav = sum(privileged * favoured) / sum(privileged)
    
    return pred_unfav / pred_fav

In [ ]:
compute_disparate_impact(privileged, favoured)

In [ ]:
X_train_no_ethnic = X_train.drop(columns=["Ethnic"])
X_test_no_ethnic = X_test.drop(columns=["Ethnic"])

categorical_features_no_ethnic = ['Agency','Gender','ScaleSet','LegalStatus','CustodyStatus', 'MaritalStatus','DisplayText']

In [ ]:
ohe = OneHotEncoder(sparse=False)

preprocessing_steps = [('categorical_transformation_no_ethnic', ohe, categorical_features_no_ethnic)]

transformer_no_ethnic = ColumnTransformer(preprocessing_steps, remainder = 'passthrough')

pipeline_steps = [('preprocess', transformer_no_ethnic),
                  ('scaler', StandardScaler()),
                  ('DT', DecisionTreeClassifier(random_state = 42))]

pipe_no_ethnic = Pipeline(pipeline_steps)

pipe_no_ethnic.fit(X_train_no_ethnic, y_train)

In [ ]:
def compute_score_di(pipe, X_test, y_test, privileged_test):

    score = pipe.score(X_test, y_test)
    di = compute_disparate_impact(privileged_test, ~pipe.predict(X_test))
    
    return score, di

In [ ]:
privileged = X_test["Ethnic"] == "Caucasian"

score_no_ethnic, di_no_ethnic = compute_score_di(pipe_no_ethnic, X_test_no_ethnic, y_test, privileged)

In [ ]:
print(score_no_ethnic), print(di_no_ethnic)

In [ ]:
def compute_reweighting_weights(privileged, favoured):
    
    w_p_fav = (sum(favoured) * sum(privileged)) / (len(favoured) * sum(favoured * privileged))
    
    w_p_unfav = (sum(~favoured) * sum(privileged)) / (len(favoured) * sum(~favoured * privileged))
    
    w_up_fav = (sum(favoured) * sum(~privileged)) / (len(favoured) * sum(favoured * ~privileged))
    
    w_up_unfav = (sum(~favoured) * sum(~privileged)) / (len(favoured) * sum(~favoured * ~privileged))
    
    weights = {"w_p_fav": w_p_fav, "w_p_unfav": w_p_unfav, "w_up_fav": w_up_fav, "w_up_unfav": w_up_unfav}
    
    return weights

In [ ]:
favoured = ~pipe.predict(X_test)
privileged = X_test["Ethnic"] == "Caucasian"

In [ ]:
compute_reweighting_weights(privileged, favoured)

In [ ]:
def generate_sample_weight(privileged, favoured, w_p_fav, w_p_unfav, w_up_fav, w_up_unfav):
    
    condlist = [(privileged == 1) & (favoured == 1), (privileged == 1) & (favoured == 0),
                (privileged == 0) & (favoured == 1), (privileged == 0) & (favoured == 0)]
    
    choicelist = [w_p_fav,w_p_unfav,w_up_fav,w_up_unfav]
    
    return np.select(condlist,choicelist)

In [ ]:
favoured = ~pipe.predict(X_test)
privileged = X_test['Ethnic'] == 'Caucasian'

In [ ]:
weights = compute_reweighting_weights(privileged, favoured)
weights

In [ ]:
test = generate_sample_weight(privileged, 
                       favoured, 
                       weights['w_p_fav'], 
                       weights['w_p_unfav'], 
                       weights['w_up_fav'], 
                       weights['w_up_unfav'])
test

In [ ]:
def apply_reweighting(pipe, X_train, y_train, privileged_train, favoured_train, X_test, y_test, privileged_test):
    
    weights = compute_reweighting_weights(privileged_train, favoured_train)
    
    w_p_fav = weights['w_p_fav']
    w_p_unfav = weights['w_p_unfav']
    w_up_fav = weights['w_up_fav']
    w_up_unfav = weights['w_up_unfav']
    
    w_train = generate_sample_weight(privileged_train, favoured_train, w_p_fav, w_p_unfav, w_up_fav, w_up_unfav)
    pipe.fit(X_train, y_train, DT__sample_weight = w_train)
    
    return compute_score_di(pipe, X_test, y_test, privileged_test)

In [ ]:
score_rw, di_rw = apply_reweighting(pipe, X_train, y_train, X_train['Ethnic'] == 'Caucasian',
                                   y_train == False, X_test, y_test, X_test['Ethnic'] == 'Caucasian')

print(score_rw, di_rw)